In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
from send2trash import send2trash
from itertools import combinations
from itertools import product
from collections import defaultdict
import unicodedata
from datetime import datetime 
from webdriver_manager.chrome import ChromeDriverManager
import os
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
import numpy as np
import time

def filtering(url1, url2):
    # URL for finviz page with filters
    url = url1

    # Creating header to attain requests
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    # Created html for scraping
    html = soup(webpage, "html.parser")

    overview_df = pd.read_html(str(html), attrs = {'class': 'table-light'})[0]

    # Processing finviz overview table
    overview_df.columns = overview_df.iloc[0]

    overview_df = overview_df.iloc[1:, :]

    overview_df = overview_df.drop(columns = ["No."])

    overview_df = overview_df.set_index("Ticker")



    url = url2

    # Creating header to attain requests
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    # Created html for scraping
    html = soup(webpage, "html.parser")

    financial_df = pd.read_html(str(html), attrs = {'class': 'table-light'})[0]

    # Processing finviz financial table
    financial_df.columns = financial_df.iloc[0]

    financial_df = financial_df.iloc[1:, :]

    financial_df = financial_df.drop(columns = ["No."])

    financial_df = financial_df.set_index("Ticker")

    df = pd.concat([overview_df, financial_df], axis = "columns", join = "inner")

    try:
        df_compare = pd.read_csv("filter_company_2.0.csv")
        ticker_list = set(list(df["Ticker"]))

        ticker_compare_list = set(list(df_compare["Ticker"]))

        print("Difference :", ticker_list - ticker_compare_list)
    except:
        pass

    df.to_csv("filter_company_2.0.csv")

import re
import json
import pandas as pd
def clean_ads_df(df):
    header = list(df.columns)
    
    array = []
    for num in range(len(df)):
        row = list(df.iloc[num, :].values)
        
        if not re.findall("[a-zA-Z]", row[0]):
            array.append(row)
            
    df_clean = pd.DataFrame(array, columns = header)
    df_clean["Date"] = pd.to_datetime(df_clean["Date"], format = '%m/%d/%Y')
    return df_clean

import numpy as np
def extract_price_target(df):
    price_clean_list = []
    
    price_list = list(df["Price Target"])
    for price_raw in price_list:
        if type(price_raw) == str:
            if re.findall("➝", str(price_raw)):
                price_split = price_raw.split()
                price_dollar = price_split[-1]

                price_dollar = re.sub(",", "", price_dollar)
                price = float(price_dollar[1:]) 
                price_clean_list.append(price)
            else:
                price_raw = re.sub(",", "", price_raw)
                price = float(price_raw[1:])
                price_clean_list.append(price)
     
    median_price = round(np.median(price_clean_list),2)
    mean_price = round(np.mean(price_clean_list),2)
    
    return median_price, mean_price


def median_mean_price_creator(html_earnings, html_price):
    try:
        earnings_df = pd.read_html(str(html_earnings), attrs = {'id': 'earnings-history'})[0]

        earnings_date_list = list(earnings_df.Date)

        last_report_date_raw = earnings_date_list[1]
        last_report_date = pd.to_datetime(pd.to_datetime(last_report_date_raw).strftime('%m/%d/%Y'))

        price_df = pd.read_html(str(html_price), attrs = {'id': 'history-table'})[0]

        price_df = clean_ads_df(price_df)

        price_df_after_report = price_df[price_df["Date"] >= last_report_date]
    
        if len(price_df_after_report) > 0:
            median_target_price, mean_target_price = extract_price_target(price_df_after_report)
        else:
            median_target_price, mean_target_price = f"No analyst ratings after last report date: {last_report_date}", f"No analyst ratings after last report date: {last_report_date}"
    except:
        median_target_price, mean_target_price = f"No tables found"
        
    return median_target_price, mean_target_price

import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
import numpy as np
import time

def scrape_html_earnings(ticker):
    # URL for each tickers market index page
    try:
        html_list = []
        for url in [f"https://www.marketbeat.com/stocks/NASDAQ/{ticker.upper()}/Earnings/",
                   f"https://www.marketbeat.com/stocks/NASDAQ/{ticker.upper()}/price-target/"]:
            # Creating header to attain requests
            req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req).read()

            # Created html for scraping
            html = soup(webpage, "html.parser")

            html_list.append(html)

        median_target_price, mean_target_price = median_mean_price_creator(html_list[0], html_list[1])
    except:
        html_list = []
        for url in [f"https://www.marketbeat.com/stocks/NYSE/{ticker.upper()}/Earnings/",
                   f"https://www.marketbeat.com/stocks/NYSE/{ticker.upper()}/price-target/"]:
            # Creating header to attain requests
            req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req).read()

            # Created html for scraping
            html = soup(webpage, "html.parser")

            html_list.append(html)

        median_target_price, mean_target_price = median_mean_price_creator(html_list[0], html_list[1])
    
    return median_target_price, mean_target_price

def median_mean_multiple_tickers(ticker_list, filter_ticker):
    filter_company = pd.read_csv("filter_company_2.0.csv")
    filter_company_list_name, filter_company_successful_eval = [], []
    for ticker in filter_ticker:
        company_name = filter_company[filter_company["Ticker"] == ticker]["Company"].values[0]
        filter_company_list_name.append(company_name)
        
        if ticker in ticker_list:
            filter_company_successful_eval.append(True)
        else:
            filter_company_successful_eval.append(False)
    
    
    median_list, mean_list = [], []
    for ticker in filter_ticker:
        median_target_price, mean_target_price = scrape_html_earnings(ticker)
        
        mean_list.append(mean_target_price)
        median_list.append(median_target_price)

    df = pd.DataFrame({"Ticker":filter_ticker, "Company":filter_company_list_name, "Mean":mean_list, "Median":median_list, "Successful evaluation": filter_company_successful_eval})
    df.to_csv("Analyst_rating_median.csv")
    display(df)
    return df

import hvplot.pandas
import panel as pn

def clean_up_df(df):
    df = df.reset_index()
    column_list = list(df.columns)
    
    clean_values_dict = {}
    for column in column_list:
        column_values = list(df[column])
        
        clean_values = []
        for value in column_values:
            if type(value) != str:
                clean_values.append(value)
                
            # Negative number identifying
            elif re.findall("\(|\)", value):
                remove_para = re.sub("\(|\)|%|,", "", str(value))
                clean_value = float(f"-{remove_para}")
                
                clean_values.append(clean_value)
                
            else:
                remove_para = re.sub("\(|\)|%|,", "", str(value))
                clean_value = float(f"{remove_para}")
                
                clean_values.append(clean_value)
                
        clean_values_dict[column] = clean_values
    
    return pd.DataFrame(clean_values_dict).set_index('year')
    
# Valuating companies
def past_growth(ticker, driver, time_to_sleep):
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')

    driver.get('https://stockrow.com/'+ str(ticker) +'/financials/income/annual')
    time.sleep(time_to_sleep)

    ticker_dict = {}
    # STR NAMES REV
    ticker_rev = 'rev'
    ticker_rev_per = 'rev(%)'

    # STR NAMES OPE
    ticker_ope = 'ope'
    ticker_ope_per = 'ope(%)'

    # STR NAMES EPS
    ticker_eps = 'eps'
    ticker_eps_per = 'eps(%)'

    row_label_num = list(range(10, 200, 9)) # num of data which are labels
    # BI important signals of a healthy company
    num_label_list = ['Revenue', 'Revenue Growth', 'Operating Income', 'EPS (Diluted)']
    num_label_growth_list = ['Operating Income Growth', 'EPS Growth (diluted)']
    #num_label_dict = {'Revenue': ticker_rev, 'Revenue Growth': ticker_rev_per, 'Operating Income': ticker_ope, 'EPS (Diluted)': ticker_eps, 'Operating Income Growth': ticker_ope_per, 'EPS Growth (diluted)': ticker_eps_per}

    years_list = []
    for num in range(3, 8):
        dat = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[4]/div/div[3]/div/div/div[4]/div/div/div['+str(num)+']')
        years_list.append(dat.text)
    
    array_dat = [] # contains lists of data
    for num in row_label_num:
        try:
            dat = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[4]/div/div[3]/div/div/div[4]/div/div/div['+str(num)+']').text
            if dat in num_label_list:
                lists_dat = []
                for num_dat in range(num + 2, num + 7):
                    dat = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[4]/div/div[3]/div/div/div[4]/div/div/div['+str(num_dat)+']').text
                    lists_dat.append(dat)
                array_dat.append(lists_dat)
        except:
            break

    driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[3]/div/div[1]/a[5]').click()
    time.sleep(7)

    for num in row_label_num:
        try:
            dat = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[4]/div/div[3]/div/div/div[4]/div/div/div['+str(num)+']').text
            if dat in num_label_growth_list:
                lists_dat = []
                for num_dat in range(num + 2, num + 7):
                    dat = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[4]/div/div[3]/div/div/div[4]/div/div/div['+str(num_dat)+']').text
                    lists_dat.append(dat)
                array_dat.append(lists_dat)
        except:
            break


    labels = [ticker_rev, ticker_rev_per, ticker_ope, ticker_eps, ticker_ope_per, ticker_eps_per]

    for dat_list, label in zip(array_dat, labels):
        ticker_dict[label] = dat_list
    ticker_dict['year'] = years_list
        
    return ticker_dict

          
        
def ticker_evaluation(df_dict, rev_growth, ope_growth, eps_growth):
    tabs = pn.Tabs(dynamic=True, tabs_location='left')
    
    good_companies = []
    for ticker in df_dict:
        df = df_dict[ticker]
        
        table = df.hvplot.table()
        
        clean_df = clean_up_df(df)
        string = ""
        check = 0
        if min(clean_df["rev(%)"]) < rev_growth:
            string += (f"{ticker} does not meet revenue growth of {rev_growth}<br>")
            check += 1
        else:
            string += (f"{ticker} meets revenue growth of {rev_growth}<br>")
            
        if min(clean_df["ope(%)"]) < ope_growth:
            string += (f"{ticker} does not meet operating income growth of {ope_growth}<br>")
            check += 1
        else:
            string += (f"{ticker} meets operating income growth of {ope_growth}<br>")
            
        if min(clean_df["eps(%)"]) < eps_growth:
            string += (f"{ticker} does not meet earnings per share growth of {eps_growth}<br>")
            check += 1
        else:
            string += (f"{ticker} meets earnings per share growth of {eps_growth}<br>")
            
        if check == 0:
            string += (f"{ticker} meets all conditions")
            good_companies.append(ticker)
            
        viz = clean_df['rev'].hvplot.bar(title = f"{ticker} Revenue", shared_axes = False) + clean_df['ope'].hvplot.bar(title = f"{ticker} Operating Income", shared_axes = False) + clean_df['eps'].hvplot.bar(title = f"{ticker} Earnings per Share", shared_axes = False)
        
        ticker_column = pn.Column(f"##{ticker}", table, string, viz)
        tabs.append((ticker, ticker_column))
        
    display(tabs)
    return good_companies, tabs


# End of functions, beginning of commands

## Filtering with Finviz
filtering("https://finviz.com/screener.ashx?v=111&f=an_recom_buybetter,fa_epsyoy_o10,fa_epsyoy1_o10,fa_estltgrowth_o5,fa_roa_o10,fa_roe_o15,geo_usa,idx_sp500,sh_avgvol_o1000,sh_instown_o60",
         "https://finviz.com/screener.ashx?v=161&f=an_recom_buybetter,fa_epsyoy_o10,fa_epsyoy1_o10,fa_estltgrowth_o5,fa_roa_o10,fa_roe_o15,geo_usa,idx_sp500,sh_avgvol_o1000,sh_instown_o60")

filter_company = pd.read_csv("filter_company_2.0.csv") 
filter_ticker = list(filter_company["Ticker"])

## Evaluating with company growth metrics
# Initiate driver
driver = webdriver.Chrome(ChromeDriverManager().install())

all_company_list = []
for ticker_company in filter_ticker:
    try:
        dat = past_growth(ticker_company, driver, 3)
        all_company_list.append(dat)
    except:
        dat = past_growth(ticker_company, driver, 4)
        all_company_list.append(dat)

# Quit driver
driver.quit()

df_dict = {}
for company, ticker in zip(all_company_list, filter_ticker):
    df = pd.DataFrame(company)
    
    # Sort years by ascending order
    df = df.sort_values(by = ["year"], ascending = [True])

    # Set year column to a datetime variable
    df["year"] = pd.to_datetime(df["year"], format = '%Y/%m/%d')
    df = df.set_index("year")
    
    # Add to the dictionary
    df_dict[ticker] = df
    
evaluated_tickers, panel = ticker_evaluation(df_dict, 5, 5, 10)

panel.save('evaluation_panel.html', embed=True, embed_json=True)

## Target price analysis
target_price_df = median_mean_multiple_tickers(evaluated_tickers, filter_ticker)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST driver version for 101.0.4951
[WDM] - Driver [/Users/jordandass/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache
/var/folders/16/t083_7x94m532lfsy9y_0dfw0000gn/T/ipykernel_1583/3214993147.py:382: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Tabs(dynamic=True, tabs_location='left')
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [2] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [3] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [4] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [5] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [6] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [7] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [8] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [9] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [10] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [11] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [12] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [13] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [14] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [15] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [16] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [17] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [18] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [19] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)

,Ticker,Company,Mean,Median,Successful evaluation
0,AMAT,"Applied Materials, Inc.",141.142857,134.0,False
1,AMD,"Advanced Micro Devices, Inc.",140.714286,140.0,False
2,ANET,"Arista Networks, Inc.",182.500000,146.0,False
3,CDNS,"Cadence Design Systems, Inc.",197.750000,198.0,False
4,CTSH,Cognizant Technology Solutions Corporation,91.500000,91.5,False
5,EL,The Estee Lauder Companies Inc.,297.333333,305.0,False
6,EPAM,"EPAM Systems, Inc.",425.000000,410.0,False
7,ETSY,"Etsy, Inc.",133.000000,130.0,False
8,EW,Edwards Lifesciences Corporation,126.000000,126.0,False
9,FB,"Meta Platforms, Inc.",288.583333,292.5,False


In [8]:
def median_mean_multiple_tickers(ticker_list, filter_ticker):
    filter_company = pd.read_csv("filter_company_2.0.csv")
    filter_company_list_name, filter_company_successful_eval = [], []
    for ticker in filter_ticker:
        company_name = filter_company[filter_company["Ticker"] == ticker]["Company"].values[0]
        filter_company_list_name.append(company_name)
        
        if ticker in ticker_list:
            filter_company_successful_eval.append(True)
        else:
            filter_company_successful_eval.append(False)
    
    
    median_list, mean_list = [], []
    for ticker in filter_ticker:
        median_target_price, mean_target_price = scrape_html_earnings(ticker)
        
        mean_list.append(mean_target_price)
        median_list.append(median_target_price)

    df = pd.DataFrame({"Ticker":filter_ticker, "Company":filter_company_list_name, "Mean":mean_list, "Median":median_list, "Successful evaluation": filter_company_successful_eval})
    df.to_csv("Analyst_rating_median.csv")
    display(df)
    return df

## Target price analysis
target_price_df = median_mean_multiple_tickers(evaluated_tickers, filter_ticker)

,Ticker,Company,Mean,Median,Successful evaluation
0,AMAT,"Applied Materials, Inc.",141.142857,134.0,False
1,AMD,"Advanced Micro Devices, Inc.",140.714286,140.0,False
2,ANET,"Arista Networks, Inc.",182.500000,146.0,False
3,CDNS,"Cadence Design Systems, Inc.",197.750000,198.0,False
4,CTSH,Cognizant Technology Solutions Corporation,91.500000,91.5,False
5,EL,The Estee Lauder Companies Inc.,297.333333,305.0,False
6,EPAM,"EPAM Systems, Inc.",425.000000,410.0,False
7,ETSY,"Etsy, Inc.",133.000000,130.0,False
8,EW,Edwards Lifesciences Corporation,126.000000,126.0,False
9,FB,"Meta Platforms, Inc.",288.583333,292.5,False


In [32]:
pane = pn.panel('evaluation_panel.html')

pane

Markdown(str)

In [27]:
from lxml import html
import requests

url =  f"https://stockrow.com/AAPL/financials/income/annual"
page = requests.get(url)
tree = html.fromstring(page.content)


In [29]:
tree.xpath('//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[4]/div/div[3]/div/div/div[4]/div/div/div[3]')

[]

In [8]:
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
import numpy as np
import time

def filtering(url1, url2):
    # URL for finviz page with filters
    url = url1

    # Creating header to attain requests
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    # Created html for scraping
    html = soup(webpage, "html.parser")

    overview_df = pd.read_html(str(html), attrs = {'class': 'table-light'})[0]

    # Processing finviz overview table
    overview_df.columns = overview_df.iloc[0]

    overview_df = overview_df.iloc[1:, :]

    overview_df = overview_df.drop(columns = ["No."])

    overview_df = overview_df.set_index("Ticker")



    url = url2

    # Creating header to attain requests
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    # Created html for scraping
    html = soup(webpage, "html.parser")

    financial_df = pd.read_html(str(html), attrs = {'class': 'table-light'})[0]

    # Processing finviz financial table
    financial_df.columns = financial_df.iloc[0]

    financial_df = financial_df.iloc[1:, :]

    financial_df = financial_df.drop(columns = ["No."])

    financial_df = financial_df.set_index("Ticker")

    df = pd.concat([overview_df, financial_df], axis = "columns", join = "inner")

    try:
        df_compare = pd.read_csv("filter_company_2.0.csv")
        ticker_list = set(list(df["Ticker"]))

        ticker_compare_list = set(list(df_compare["Ticker"]))

        print("Difference :", ticker_list - ticker_compare_list)
    except:
        pass

    df.to_csv("filter_company_2.0.csv")

    display(df)
    
filtering("https://finviz.com/screener.ashx?v=111&f=an_recom_buybetter,fa_epsyoy_o10,fa_epsyoy1_o10,fa_estltgrowth_o5,fa_roa_o10,fa_roe_o15,geo_usa,idx_sp500,sh_avgvol_o1000,sh_instown_o60",
         "https://finviz.com/screener.ashx?v=161&f=an_recom_buybetter,fa_epsyoy_o10,fa_epsyoy1_o10,fa_estltgrowth_o5,fa_roa_o10,fa_roe_o15,geo_usa,idx_sp500,sh_avgvol_o1000,sh_instown_o60")



,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume,Market Cap,...,Quick R,LTDebt/Eq,Debt/Eq,Gross M,Oper M,Profit M,Earnings,Price,Change,Volume
Ticker,,,,,,,,,,,,,,,,,,,,,
AMAT,"Applied Materials, Inc.",Technology,Semiconductor Equipment & Materials,USA,97.95B,14.22,106.59,-3.04%,6751639,97.95B,...,1.80,0.46,0.00,47.50%,27.70%,23.70%,May 19/a,106.59,-3.04%,6751639
AMD,"Advanced Micro Devices, Inc.",Technology,Semiconductors,USA,152.73B,34.00,91.16,-4.11%,123058689,152.73B,...,2.00,0.03,0.03,48.50%,20.80%,18.00%,May 03/a,91.16,-4.11%,123058689
ANET,"Arista Networks, Inc.",Technology,Computer Hardware,USA,30.27B,34.15,99.69,-2.55%,1970246,30.27B,...,3.20,0.00,0.00,63.60%,31.60%,29.50%,May 02/a,99.69,-2.55%,1970246
CDNS,"Cadence Design Systems, Inc.",Technology,Software - Application,USA,38.96B,53.77,144.00,-2.46%,1482173,38.96B,...,1.70,0.13,0.00,89.80%,28.30%,23.60%,Apr 25/a,144.00,-2.46%,1482173
CTSH,Cognizant Technology Solutions Corporation,Technology,Information Technology Services,USA,36.94B,17.13,71.40,-0.61%,2848787,36.94B,...,2.20,0.05,0.05,36.90%,15.20%,11.60%,May 04/a,71.40,-0.61%,2848787
EL,The Estee Lauder Companies Inc.,Consumer Defensive,Household & Personal Products,USA,82.29B,25.36,232.06,-1.92%,1662291,82.29B,...,1.20,0.84,0.89,76.50%,18.40%,18.50%,May 03/b,232.06,-1.92%,1662291
EPAM,"EPAM Systems, Inc.",Technology,Information Technology Services,USA,18.02B,39.50,308.87,-1.84%,848505,18.02B,...,3.30,0.01,0.02,33.80%,13.60%,11.10%,May 05/b,308.87,-1.84%,848505
ETSY,"Etsy, Inc.",Consumer Cyclical,Internet Retail,USA,9.03B,23.70,70.21,-7.70%,5963397,9.03B,...,2.70,3.52,3.53,71.00%,16.90%,18.50%,May 04/a,70.21,-7.70%,5963397
EW,Edwards Lifesciences Corporation,Healthcare,Medical Devices,USA,58.81B,38.75,94.54,-1.65%,3031531,58.81B,...,2.60,0.10,0.10,76.60%,32.60%,28.70%,Apr 26/a,94.54,-1.65%,3031531


In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
from send2trash import send2trash
from itertools import combinations
from itertools import product
from collections import defaultdict
import unicodedata
from datetime import datetime 
from webdriver_manager.chrome import ChromeDriverManager
import os
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


# Valuating companies
def past_growth(ticker, driver, time_to_sleep):
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')

    driver.get('https://stockrow.com/'+ str(ticker) +'/financials/income/annual')
    time.sleep(time_to_sleep)

    ticker_dict = {}
    # STR NAMES REV
    ticker_rev = 'rev'
    ticker_rev_per = 'rev(%)'

    # STR NAMES OPE
    ticker_ope = 'ope'
    ticker_ope_per = 'ope(%)'

    # STR NAMES EPS
    ticker_eps = 'eps'
    ticker_eps_per = 'eps(%)'

    row_label_num = list(range(10, 200, 9)) # num of data which are labels
    # BI important signals of a healthy company
    num_label_list = ['Revenue', 'Revenue Growth', 'Operating Income', 'EPS (Diluted)']
    num_label_growth_list = ['Operating Income Growth', 'EPS Growth (diluted)']
    #num_label_dict = {'Revenue': ticker_rev, 'Revenue Growth': ticker_rev_per, 'Operating Income': ticker_ope, 'EPS (Diluted)': ticker_eps, 'Operating Income Growth': ticker_ope_per, 'EPS Growth (diluted)': ticker_eps_per}

    years_list = []
    for num in range(3, 8):
        dat = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[4]/div/div[3]/div/div/div[4]/div/div/div['+str(num)+']')
        years_list.append(dat.text)
    
    array_dat = [] # contains lists of data
    for num in row_label_num:
        try:
            dat = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[4]/div/div[3]/div/div/div[4]/div/div/div['+str(num)+']').text
            if dat in num_label_list:
                lists_dat = []
                for num_dat in range(num + 2, num + 7):
                    dat = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[4]/div/div[3]/div/div/div[4]/div/div/div['+str(num_dat)+']').text
                    lists_dat.append(dat)
                array_dat.append(lists_dat)
        except:
            break

    driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[3]/div/div[1]/a[5]').click()
    time.sleep(7)

    for num in row_label_num:
        try:
            dat = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[4]/div/div[3]/div/div/div[4]/div/div/div['+str(num)+']').text
            if dat in num_label_growth_list:
                lists_dat = []
                for num_dat in range(num + 2, num + 7):
                    dat = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/section/div/div[2]/div[1]/section[4]/div/div[3]/div/div/div[4]/div/div/div['+str(num_dat)+']').text
                    lists_dat.append(dat)
                array_dat.append(lists_dat)
        except:
            break


    labels = [ticker_rev, ticker_rev_per, ticker_ope, ticker_eps, ticker_ope_per, ticker_eps_per]

    for dat_list, label in zip(array_dat, labels):
        ticker_dict[label] = dat_list
    ticker_dict['year'] = years_list
        
    return ticker_dict


filter_company = pd.read_csv("filter_company_2.0.csv")
filter_ticker = filter_company.reset_index()['Ticker']

# Initiate driver
driver = webdriver.Chrome(ChromeDriverManager().install())

all_company_list = []
for ticker_company in filter_ticker:
    dat = past_growth(ticker_company, driver, 2)
    all_company_list.append(dat)

# Quit driver
driver.quit()

df_dict = {}
for company, ticker in zip(all_company_list, filter_ticker):
    df = pd.DataFrame(company)
    
    # Sort years by ascending order
    df = df.sort_values(by = ["year"], ascending = [True])

    # Set year column to a datetime variable
    df["year"] = pd.to_datetime(df["year"], format = '%Y/%m/%d')
    df = df.set_index("year")
    
    # Add to the dictionary
    df_dict[ticker] = df


[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST driver version for 101.0.4951
[WDM] - Get LATEST driver version for 101.0.4951
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/jordandass/.wdm/drivers/chromedriver/mac64/101.0.4951.41]
/var/folders/16/t083_7x94m532lfsy9y_0dfw0000gn/T/ipykernel_841/3857400426.py:102: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [18]:
import hvplot.pandas
import panel as pn

def clean_up_df(df):
    df = df.reset_index()
    column_list = list(df.columns)
    
    clean_values_dict = {}
    for column in column_list:
        column_values = list(df[column])
        
        clean_values = []
        for value in column_values:
            if type(value) != str:
                clean_values.append(value)
                
            # Negative number identifying
            elif re.findall("\(|\)", value):
                remove_para = re.sub("\(|\)|%|,", "", str(value))
                clean_value = float(f"-{remove_para}")
                
                clean_values.append(clean_value)
                
            else:
                remove_para = re.sub("\(|\)|%|,", "", str(value))
                clean_value = float(f"{remove_para}")
                
                clean_values.append(clean_value)
                
        clean_values_dict[column] = clean_values
    
    return pd.DataFrame(clean_values_dict).set_index('year')
    
                
        
def ticker_evaluation(df_dict, rev_growth, ope_growth, eps_growth):
    tabs = pn.Tabs(dynamic=True)
    
    good_companies = []
    for ticker in df_dict:
        print(ticker)
        df = df_dict[ticker]
        
        table = df.hvplot.table()
        
        clean_df = clean_up_df(df)
        string = ""
        check = 0
        if min(clean_df["rev(%)"]) < rev_growth:
            string += (f"{ticker} does not meet revenue growth of {rev_growth}<br>")
            check += 1
        else:
            string += (f"{ticker} meets revenue growth of {rev_growth}<br>")
            
        if min(clean_df["ope(%)"]) < ope_growth:
            string += (f"{ticker} does not meet operating income growth of {ope_growth}<br>")
            check += 1
        else:
            string += (f"{ticker} meets operating income growth of {ope_growth}<br>")
            
        if min(clean_df["eps(%)"]) < eps_growth:
            string += (f"{ticker} does not meet earnings per share growth of {eps_growth}<br>")
            check += 1
        else:
            string += (f"{ticker} meets earnings per share growth of {eps_growth}<br>")
            
        if check == 0:
            string += (f"{ticker} meets all conditions")
            good_companies.append(ticker)
            
        viz = clean_df['rev'].hvplot.bar(title = f"{ticker} Revenue", shared_axes = False) + clean_df['ope'].hvplot.bar(title = f"{ticker} Operating Income", shared_axes = False) + clean_df['eps'].hvplot.bar(title = f"{ticker} Earnings per Share", shared_axes = False)
        
        ticker_column = pn.Column(f"##{ticker}", table, string, viz)
        tabs.append((ticker, ticker_column))
        
    display(tabs)
    return good_companies

ticker_evaluation(df_dict, 5, 5, 10)

AMAT
ANET
CDNS
CTSH
EL
EPAM
ETSY
EW
FB
FTNT
GOOGL
INCY
KLAC
LRCX
MA
MCO
MSFT
MU
NKE
NRG


Tabs(dynamic=True)
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [2] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [3] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [4] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [5] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [6] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [7] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [8] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [9] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [10] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [11] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [12] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [13] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [14] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [15] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [16] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [17] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [18] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)
    [19] Column
        [0] Markdown(str)
        [1] HoloViews(Table)
        [2] Markdown(str)
        [3] HoloViews(Layout)

[]